In [1]:
import os, numpy as np, scipy, pandas as pd
from lpEnergyModels.mBasicInt import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# MBasicPH class

The model extends the ```MBasicInt``` model in the following ways:
* **Power and heat markets**: We model power and heat markets with explicit equilibrium constraints in all hours of the model. We introduce this distinction by adding new symbol names rather than introduce a new "market" index. We do this because characteristics of the different energy markets differ quite a lot; thus, the advantage of being able to apply some methods to all energy markets is limited. 
* **Geography**: Generators and consumers may engage in more than one energy market, but they are tied to unique geographic locations. We have separate geographic indices, ```idxGE``` and ```idxGH```, for electricity and heat respectively.
* **Trade**: To model trade between geographic areas, we specify trade lines ```idxTrL``` index to indicate trade lines (or transmission lines) between areas. Currently, these are only used for electricity markets (district heat is not traded across significant geographic areas in the same way).
* **Plant types:** In MBasicInt the only difference between generators were captured by different parameter values (capacity, costs, variation in productivity). In this model, we introduce different types using an index ```idxPT``` to distinguish between four dominant different types: Standard electricity producing plants (```'stdE'```), standard heat producing plants (```'stdH'```), combined heat and power plants (```'CHP'```), and heat pumper (```'HP'```). This categorical distinction is used to make model generation easier and is identified using a mapping ```idxGen2PT``` (generatior idx maps to unique plant type). 

#### **A. Sets:**

The model relies on the same sets as the *MBasicInt* model, adjusts some of the sets (to include both heat and electricity variants) and adds new.

*From MBasicInt*
* ```idxF```: Set of fuel types that generators may use.
* ```idxEm```: Set of emission types included in the model.
* ```idxHr```: Set of short run states (hours).

*Sets from MBasicInt adjusted with E/H distinction:*
* ```idxGen```: Split into two subsets. ```idxGenE, idxGenH```. Note:
    * A generator can be in both ```idxGenE``` and ```idxGenH``` if it produces both (e.g. combined heat and power plants).
    * Per construction, we define the full index ```idxGen``` as the union of the two sets.
* ```idxCons```: Same procedure (split into E,H - add full index from union) 
* ```idxHVTGen```: Same procedure (split into E,H - add full index from union). 
* ```idxHVTCons```: Same procedure (split into E,H - add full index from union).


*New sets:*
* ```idxGE```, ```idxGH```: Geography indices.
* ```idxPT```: Plant types. All generators should be mapped to unique type (mapping ```idxGen2PT```). Currently includes
    * ```'stdE'```: Standard electricity producing plant. Behavior is similar to ```MBasicInt``` plants. Generator has to be mapped to unique element in geography index ```idxGE``` (mapping ```idxGen2GE```).
    * ```'stdH'```: Standard heat producing plant. Reflects similar behavior as standard electricity generating plants, but for the heat markets. Generator has to be mapped to unique element in ```idxGH```. (mapping ```idxGen2GH```)
    * ```'HP'```: Heat pump. Consumes electricity, produces heat. Generator has to be mapped to unique element in ```idxGE``` and ```idxGH```. 
    * ```'CHP'```: Combined heat and power producers. Generator has to be mapped to unique element in ```idxGE``` and ```idxGH```.
* ```idxTrL```: Trade/transmission line index that links two adjacent geographic areas with a trade capacity.
* ```idxHVTTrL```: Set of exogenously hourly variation paths, for trade/transmission lines. 

#### **B. Variables:**

*From MBasicInt:*
* ```pFuel(idxF)```: Fuel price. Default unit: €/GJ.
* ```uEm(idxF, idxEm)```: Emission intensity. Default unit: Ton emission/GJ fuel input. 
* ```VOM(idxGen)```: variable operating and maintenance costs. Default unit: €/GJ output.
* ```uFuel(idxF, idxGen)```: Fuelmix. Default unit: GJ input/GJ output.
* ```taxEm(idxEm)```: Tax on emissions. Default unit: €/ton emission output.
* ```uHrCap(idxHr, idxHVTGen)```: Parameter measuring how generating capacity varies over the year for variation type ```idxHVTGen```. Default unit: Hourly-to-maximum GJ/hour ratio. Generally, we expect ```uHrCap```$\in[0,1]$ as it measures the scale of generating capacity in hour ```idxH``` relatively to installed effect (```genCap```).
* ```uHrLoad(idxHr, idxHVTCons)```: Similar to ```uHrCap``` except this measures max load relative to peak demand throughout a year. 


*From MBasicInt, but with added distinction between E/H*.
* ```genCap(idxGen)```: Is split into one for electricity, one for heat (```genCapE```, ```genCapH```).
* ```mwp(idxCons)```: Marginal willingness to pay for consumers. Is split into one for electricity, one for heat (```mwpE```, ```mwpH```). 
* ```load(idxCons)```: Total demand for consumers if the price does not exceed their marginal willingness to pay. Default unit: GJ. (```loadE```, ```loadH```)

**Variables:**

*From MBasicInt:*
* 

**Variables:** The model requires the following data to run:

*From MBasicInt*:
* ```pFuel(idxF)```: Fuel price. Default unit: €/GJ.
* ```uEm(idxF, idxEm)```: Emission intensity. Default unit: Ton emission/GJ fuel input. 
* ```VOM(idxGen)```: variable operating and maintenance costs. Default unit: €/GJ output.
* ```uFuel(idxF, idxGen)```: Fuelmix. Default unit: GJ input/GJ output.
* ```taxEm(idxEm)```: Tax on emissions. Default unit: €/ton emission output.
* ```uHrCap(idxHr, idxHVTGen)```: Parameter measuring how generating capacity varies over the year for variation type ```idxHVTGen```. Default unit: Hourly-to-maximum GJ/hour ratio. Generally, we expect ```uHrCap```$\in[0,1]$ as it measures the scale of generating capacity in hour ```idxH``` relatively to installed effect (```genCap```).
* ```uHrLoad(idxHr, idxHVTCons)```: Similar to ```uHrCap``` except this measures max load relative to peak demand throughout a year. 


*From MBasicInt, but with added distinction between E/H*.
* ```genCap(idxGen)```: Is split into one for electricity, one for heat (```genCapE```, ```genCapH```).
* ```mwp(idxCons)```: Marginal willingness to pay for consumers. Is split into one for electricity, one for heat (```mwpE```, ```mwpH```). 
* ```load(idxCons)```: Total demand for consumers if the price does not exceed their marginal willingness to pay. Default unit: GJ. (```loadE```, ```loadH```)

*Mappings from MBasicInt:* 

* ```idxGen2HVTGen(idxGen, idxHVTGen)```: Identifies what hourly variation pattern the specific generator follows. 
* ```idxCons2HVTCons(idxCons, idxHVTCons)```: Identifies hourly variation pattern specific demand component follows.
* `` idx